In [146]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################

#Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

# # dx = 1 km; Np = 1M; Nt = 5 min
# data=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_5min.nc') #***
# parcel=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc') #***
# res='1km';t_res='5min'
# Np_str='1e6'

# dx = 1km; Np = 50M
#Importing Model Data
check=False
dir2='/home/air673/koa_scratch/'
data=xr.open_dataset(dir2+'cm1out_1km_1min.nc') #***
parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_50M.nc') #***
res='1km'; t_res='1min'; Np_str='50e6'

# # dx = 1km; Np = 100M
# #Importing Model Data
# check=False
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_1km_1min.nc') #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_100M.nc') #***
# res='1km'; t_res='1min'; Np_str='100e6'


# dx = 250 m
# #Importing Model Data
# check=False
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_250m.nc') #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_250m.nc') #***

In [134]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

In [135]:
#JOB ARRAY SETUP
job_array=False
job_array=True


if job_array==True:

    num_jobs=60 #how many total jobs are being run? i.e. array=1-100 ==> num_jobs=100 #***
    total_elements=len(data['time']) #total num of variables

    if num_jobs >= total_elements:
        raise ValueError("Number of jobs cannot be greater than or equal to total elements.")
    
    job_range = total_elements // num_jobs  # Base size for each chunk
    remaining = total_elements % num_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_jobs - 1: 
            end_job = total_elements #- 1
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()
    
    job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    if job_id==0: job_id=50
    start_job, end_job = get_job_range(job_id, num_jobs)
    index_adjust=start_job
    print(f'start_job = {start_job}, end_job = {end_job}')

start_job = 540, end_job = 551


In [136]:
if job_array==True:
    #Indexing Array with JobArray
    data=data.isel(time=slice(start_job,end_job))
    parcel=parcel.isel(time=slice(start_job,end_job))
    #(for 150_000_000 parcels use 500-1000 jobs)
if job_array==False:
    start_job=0;end_job=len(data['time']);index_adjust=0

In [112]:
# #OLD VERSION (CURRENTLY PRECALCULATED)
# def call_variables(): #***
#     variable='winterp'; w_data=data[variable].data
#     # variable='qv'; qv_data=data[variable].data # get qc data
#     variable='qc'; qc_data=data[variable].data # get qc data
#     variable='qi'; qi_data=data[variable].data # get qc data
#     qc_plus_qi=qc_data+qi_data
#     return w_data,qv_data,qc_data,qi_data,qc_plus_qi

# print('calling variables')
# [w_data,qv_data,qc_data,qi_data,qc_plus_qi]=call_variables()
# print('done')

In [113]:
# #OLD VERSION
# #CALCULATING EULERIAN VMF
# variable='rho';rho_data=data[variable].data
# # print('calculating A_c')
# w_thresh2=0.5
# qcqi_thresh=1e-6
# cond1 = w_data >= w_thresh2
# cond2 = qc_plus_qi >= qcqi_thresh
# A_c = np.where(cond1 & cond2, 1, 0)

# w_thresh1=0.1
# cond1 = w_data >= w_thresh1
# cond2 = qc_plus_qi < qcqi_thresh
# A_g = np.where(cond1 & cond2, 1, 0)

# # print('calculating VMF everywhere')
# VMF_g = rho_data*w_data*A_g
# VMF_c = rho_data*w_data*A_c

In [139]:
###############################################
#READING BACK IN
out_file = dir2 + 'Variable_Calculation/' + f'Eulerian_VMF_{res}_{t_res}.h5'

with h5py.File(out_file, 'r') as f:
    VMF_c = f['VMF_c'][start_job:end_job]  # Loads the entire A_c array into memory
    VMF_g = f['VMF_g'][start_job:end_job]  # Loads the entire A_g array into memory

In [140]:
###############################################
#READING BACK IN
out_file = dir2 + 'Variable_Calculation/' + f'Eulerian_Binary_Array_{res}_{t_res}.h5'

with h5py.File(out_file, 'r') as f:
    A_c = f['A_c'][start_job:end_job]  # Loads the entire A_c array into memory
    A_g = f['A_g'][start_job:end_job]  # Loads the entire A_g array into memory

In [116]:
##########################################################################################
#Functions
# Full Profile function makes profile together for all timesteps. AveragedProfiles funciton takes the final mean of the combined profile.

In [117]:
# #OLD VERSION
# # Thresholds
# w_thresh1 = 0.1
# w_thresh2 = 0.5
# qcqi_thresh = 1e-6

# def final_profile(vars_list, type):
#     zhs = data['zh'].values
#     profiles = {}  # Store profiles for all variables

#     # Initialize profiles for each variable
#     for var in vars_list:
#         profiles[var] = np.zeros((len(zhs), 3))  # column 1: var, column 2: counter, column 3: list of zhs
#         profiles[var][:, 2] = zhs

#     # Threshold mask
#     if type == "general":
#         where_updraft = (w_data >= w_thresh1) & (qc_plus_qi < qcqi_thresh)
#     elif type == "cloudy":
#         where_updraft = (w_data >= w_thresh2) & (qc_plus_qi >= qcqi_thresh)
        
#     t_ind, z_ind, y_ind, x_ind = np.where(where_updraft)

#     # Variable selection dictionary
#     var_data = {
#         'VMF_g': VMF_g, 'VMF_c': VMF_c, 
#     }
    
#     # Iterate over each variable in vars_list and bin the data
#     for var in vars_list:
#         masked_data = var_data[var][where_updraft]
#         np.add.at(profiles[var][:, 0], z_ind, masked_data)
#         np.add.at(profiles[var][:, 1], z_ind, 1)

#     return profiles

In [141]:
# Thresholds
w_thresh1 = 0.1
w_thresh2 = 0.5
qcqi_thresh = 1e-6

def final_profile(vars_list, type):
    zhs = data['zh'].values
    profiles = {}  # Store profiles for all variables

    # Initialize profiles for each variable
    for var in vars_list:
        profiles[var] = np.zeros((len(zhs), 3))  # column 1: var, column 2: counter, column 3: list of zhs
        profiles[var][:, 2] = zhs

    # Threshold mask
    if type == "general":
        where_updraft = (A_g==True)
    elif type == "cloudy":
        where_updraft = (A_c==True)
        
    t_ind, z_ind, y_ind, x_ind = np.where(where_updraft)

    # Variable selection dictionary
    var_data = {
        'VMF_g': VMF_g, 'VMF_c': VMF_c, 
    }
    
    # Iterate over each variable in vars_list and bin the data
    for var in vars_list:
        masked_data = var_data[var][where_updraft]
        np.add.at(profiles[var][:, 0], z_ind, masked_data)
        np.add.at(profiles[var][:, 1], z_ind, 1)

    return profiles

In [142]:
##########################################################################################
#Running

In [143]:
dir2=dir+'Project_Algorithms/Domain_Profiles/'

if job_array==True:
    print(f"job_id = {job_id}")
for type in ["general","cloudy"]:
    print(f"working on {type} type\n")
    
    vars_list = ['VMF_g', 'VMF_c']
    profiles = final_profile(vars_list, type)
    
    #Saving eulerian_profiles
    import h5py
    
    if type == "general":
        output_file = dir2+f'job_out/general_eulerian_eulerian_VMF_profiles_{res}_{t_res}_{Np_str}'
    elif type == "cloudy":
        output_file = dir2+f'job_out/cloudy_eulerian_eulerian_VMF_profiles_{res}_{t_res}_{Np_str}'
    if job_array==True:
        output_file+=f'_{job_id}.h5'
    elif job_array==False:
        output_file+=f'.h5'
    
    with h5py.File(output_file, 'w') as f:
        for var in profiles:
            profile_var = profiles[var]
            f.create_dataset(f'profile_{var}', data=profile_var, compression="gzip")

job_id = 50
working on general type

working on cloudy type



In [145]:
check_memory(globals())

Top 10 objects with highest memory usage
{'qc_plus_qi': '1852.21 MB', 'cond1': '463.05 MB', 'cond2': '463.05 MB', 'VMF_g': '306.38 MB', 'VMF_c': '306.38 MB', 'A_c': '38.3 MB', 'A_g': '38.3 MB', 'NamespaceMagics': '0.0 MB', 'Normalize': '0.0 MB', 'MaxNLocator': '0.0 MB'}

3.46767 GB in use overall


In [147]:
#########################################
#RECOMBINE SEPERATE JOB_ARRAYS AFTER
recombine=False #KEEP FALSE WHEN JOB ARRAY IS RUNNING
# recombine=True

In [148]:
if recombine==True:
    dir2=dir+'Project_Algorithms/Domain_Profiles/'
    
    types=['general','cloudy']
    for type in types:
        #MAKING OUTPUT FILE PATH
        if type == "general":
            output_file =dir2+f'job_out/general_eulerian_eulerian_VMF_profiles_{res}_{t_res}_{Np_str}.h5'
        elif type == "cloudy":
            output_file = dir2+f'job_out/cloudy_eulerian_eulerian_VMF_profiles_{res}_{t_res}_{Np_str}.h5'
        
        #MAKING PROFILES DICTIONARY
        zhs = data['zh'].values
        profiles = {}  # Store profiles for all variables
        vars_list = ['VMF_g', 'VMF_c']
        for var in vars_list:
            profiles[var] = np.zeros((len(zhs), 3))  # column 1: var, column 2: counter, column 3: list of zhs
            profiles[var][:, 2] = zhs 
        
        num_jobs=60
        for job_id in np.arange(1,num_jobs+1):
            if np.mod(job_id,10)==0: print(f"job_id = {job_id}")
    
            #CALLING IN DATA
            if type == "general":
                input_file = dir2+f'job_out/general_eulerian_eulerian_VMF_profiles_{res}_{t_res}_{Np_str}_{job_id}.h5' 
            elif type == "cloudy":
                input_file = dir2+f'job_out/cloudy_eulerian_eulerian_VMF_profiles_{res}_{t_res}_{Np_str}_{job_id}.h5' 
    
            #COMPILING PROFILES
            with h5py.File(input_file, 'r') as f:
                for var in vars_list:  
                    profiles[var][:,0:1+1]+=f[f'profile_{var}'][:,0:1+1]
        
        #SAVING INTO FINAL FORM
        with h5py.File(output_file, 'w') as f:
            for var in profiles:
                profile_var = profiles[var]
                f.create_dataset(f'profile_{var}', data=profile_var, compression="gzip")

job_id = 10
job_id = 20
job_id = 30
job_id = 40
job_id = 50
job_id = 60
job_id = 10
job_id = 20
job_id = 30
job_id = 40
job_id = 50
job_id = 60
